## Import Libraries

- We import necessary libraries including RandomForestClassifier, SHAP, metrics, and GridSearchCV.

In [11]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from collections import Counter

import pickle

## Load Prepared Data (Encoded + Feature Engineered)

In [2]:
data_path = "D:/Sajid/Chrun_Prediction/data/processed/final_encoded_data.csv"
df = pd.read_csv(data_path)

In [3]:
# Split Data into train and test data
X = df.drop(columns=["Churn"])
y = df["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## Initial RF Model for SHAP

- We train an initial Random Forest model to calculate SHAP values for feature importance, focusing on the minority class (churn = 1).

In [4]:
# Train Initial RF Model
rf_initial = RandomForestClassifier(n_estimators=500, random_state=42, class_weight="balanced")
rf_initial.fit(X_train, y_train)

# Select a subset of data for SHAP computation (to reduce computation time)
subset_size = 1000  # Adjust as needed
sample_indices = np.random.choice(X_train.shape[0], subset_size, replace=False)
X_train_sample = X_train.iloc[sample_indices]

# Compute SHAP Values on subset
explainer = shap.TreeExplainer(rf_initial)
shap_values = explainer.shap_values(X_train_sample)

# Convert SHAP values into a 1D importance array --> 
#shap_values.shape = (total_samples, total_features, total_classes) --> shap_values.shape = (1000, 35, 2)
#mean(axis=0) → Averages over all samples to get feature importance
feature_importance = np.abs(shap_values).mean(axis=0)[:, 1]  # [:, 1] Selects SHAP values only for Class 1


# Create a DataFrame of feature importance
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

# Sort features by importance (highest first)
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

display(feature_importance_df)


,Feature,Importance
9,is_long_term_contract,0.068139
4,tenure,0.066881
8,TotalCharges,0.051918
19,InternetService_Fiber optic,0.048589
7,MonthlyCharges,0.041605
34,Contract_Two year,0.033076
36,PaymentMethod_Electronic check,0.029414
12,is_tech_dependent,0.020821
22,OnlineSecurity_Yes,0.015376
6,PaperlessBilling,0.012471


## Select Features Based on SHAP Importance Threshold
- We select features with SHAP importance greater than 0.02 for further modeling.

In [5]:
# Select Features with SHAP Importance > Threshold (e.g., 0.01)
selected_features = feature_importance_df[feature_importance_df['Importance'] > 0.02]['Feature'].tolist()
print(f"Selected {len(selected_features)} features based on SHAP")

Selected 8 features based on SHAP


## Drop Highly Correlated Features (among selected)

- We remove highly correlated features (corr > 0.8), keeping the more important one based on shape score.

In [6]:
# Subset X_train to only include SHAP-selected features
X_shap_selected = X_train[selected_features].copy()

# Compute the absolute correlation matrix
corr_matrix = X_shap_selected.corr().abs()

# Select upper triangle of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop features with high correlation (> 0.8) and lower SHAP importance
features_to_drop = []

for col in upper_tri.columns:
    high_corr = upper_tri[col][upper_tri[col] > 0.8].index.tolist()
    for correlated_feature in high_corr:
        # Compare SHAP importance of the two features
        imp_col = feature_importance_df.loc[feature_importance_df['Feature'] == col, 'Importance'].values[0]
        imp_corr = feature_importance_df.loc[feature_importance_df['Feature'] == correlated_feature, 'Importance'].values[0]

        if imp_col < imp_corr:
            features_to_drop.append(col)
        else:
            features_to_drop.append(correlated_feature)

# Remove duplicates
features_to_drop = list(set(features_to_drop))

# Final cleaned feature list
final_features = [f for f in selected_features if f not in features_to_drop]

print(f"\n Final Features After SHAP + Multicollinearity Filtering: {len(final_features)}")
print(final_features)



 Final Features After SHAP + Multicollinearity Filtering: 7
['is_long_term_contract', 'tenure', 'InternetService_Fiber optic', 'MonthlyCharges', 'Contract_Two year', 'PaymentMethod_Electronic check', 'is_tech_dependent']


## GridSearchCV for Random Forest Hyperparameter Tuning

- We perform cross-validated grid search using F1-score as the evaluation metric, testing over parameters like:
- `n_estimators`
- `max_depth`
- `min_samples_split`
- `min_samples_leaf`
- `max_features`
- `class_weight`, etc.

In [7]:
# Define Parameter Grid for Grid Search
param_grid = {
    'n_estimators': [200, 500, 1000],    # Number of trees in the forest
    'max_depth': [10, 20, None],             # Depth of trees
    'min_samples_split': [ 5, 10],         # Min samples to split an internal node
    'min_samples_leaf': [2, 4],           # Min samples required to be a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features considered for split
    #'bootstrap': [True, False],              # Whether to use bootstrap samples --> a technique where random samples are drawn with replacement from the dataset to train each decision tree. it actually introduces randomness, making the model more robust. --> if all trees are created using same data then they are likely to bi highly corr causing overfitting
    #'criterion': ['gini', 'entropy'] ,        # Split criterion
    'class_weight': ['balanced', {0: 1, 1: 1.25}, {0: 1, 1: 1.5},{0: 1, 1: 2}] 
}
# Perform Grid Search on Selected Features
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                       param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=2)

rf_grid.fit(X_train[final_features], y_train)


Fitting 3 folds for each of 432 candidates, totalling 1296 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': ['balanced', {0: 1, 1: 1.25},
                                          {0: 1, 1: 1.5}, {0: 1, 1: 2}],
                         'max_depth': [10, 20, None],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [2, 4],
                         'min_samples_split': [5, 10],
                         'n_estimators': [200, 500, 1000]},
             scoring='f1', verbose=2)

## Train Final Random Forest Model
Using the best hyperparameters from GridSearchCV, we fit the final Random Forest model on the selected features.

In [9]:

# Train Final RF with Best Hyperparameters
best_params = rf_grid.best_params_
print(f"Best Parameters from Grid Search: {best_params}")

rf_final = RandomForestClassifier(**best_params, random_state=42)
rf_final.fit(X_train[final_features], y_train)



Best Parameters from Grid Search: {'class_weight': {0: 1, 1: 2}, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1000}


RandomForestClassifier(class_weight={0: 1, 1: 2}, max_depth=10,
                       min_samples_leaf=4, min_samples_split=10,
                       n_estimators=1000, random_state=42)

## Evaluate Final Model Performance
We evaluate performance using:
- ROC AUC Score
- Classification Report (Precision, Recall, F1-score)
- Threshold optimization based on F1-score

In [10]:
# Evaluate on Test Set
#y_pred = rf_final.predict(X_test[selected_features])
y_pred_proba = rf_final.predict_proba(X_test[final_features])[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba) ### generate array of precison, recall and threshold
f1_scores = 2 * (precision * recall) / (precision + recall)


# to get threshold with best f1 score
best_threshold = thresholds[np.argmax(f1_scores)] 
print(f'\nbest threshold with max f1_score: {best_threshold}')


y_pred = (y_pred_proba > best_threshold).astype(int)  # Adjust threshold here





print("Final Model Accuracy:", accuracy_score(y_test, y_pred))
print("Final Test Set AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


best threshold with max f1_score: 0.3471992579906119
Final Model Accuracy: 0.7263681592039801
Final Test Set AUC: 0.8319843558298089
              precision    recall  f1-score   support

           0       0.92      0.69      0.79      1033
           1       0.49      0.83      0.62       374

    accuracy                           0.73      1407
   macro avg       0.70      0.76      0.70      1407
weighted avg       0.81      0.73      0.74      1407



## Save final Random Forest model

In [12]:
pickle.dump(rf_final, open("D:/Sajid/Chrun_Prediction/deployment/final_rf_model.pkl", "wb"))
print("Random Forest model saved as final_rf_model.pkl")

Random Forest model saved as final_rf_model.pkl


## Save final SHAP-selected features after filtering

In [15]:
#Save final SHAP-selected features after filtering
import joblib
joblib.dump(final_features, "D:/Sajid/Chrun_Prediction/deployment/shap_selected_features.pkl")
print("Saved SHAP-selected features as shap_selected_features.pkl")

Saved SHAP-selected features as shap_selected_features.pkl
